# Pneumonia identification from X-ray images

**Authors**: Pedro Bueso-Inchausti, Sara Sanchez, Ignacio Taguas

**Subject**: Big Data Engineering

**Date**: 26-01-2020

## 1. Objective

In this excercise, we will be using deep neural networks to classify X-ray images as belonging to "*Normal*" and "*Pneumonia*". Chest X-ray imaging is commonly used as a method for diagnosis, but rapid interpretation is not always available. This is particularly true for low-resources settings where pneumonia has the highest incidence and mortality rate. This kind of classificators are, therefore, of great importance.

## 2. Prerequisites

We import everything that is needed from pyspark. We initialise the Spark Context and the SQL Context.

In [ ]:
import os; os.environ["PYSPARK_SUBMIT_ARGS"] = "--packages databricks:spark-deep-learning:1.5.0-spark2.4-s_2.11 pyspark-shell"
import findspark; findspark.init()
from pyspark.sql import SparkSession; spark = SparkSession.builder.appName("Pneumonia").getOrCreate(); sc = spark.sparkContext
from pyspark.sql import SQLContext; sqlc = SQLContext(sc)
from pyspark.sql.functions import lit
from pyspark.ml import Pipeline
from pyspark.ml.image import ImageSchema
from pyspark.ml.classification import LogisticRegression, MultilayerPerceptronClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from sparkdl import DeepImageFeaturizer

We import the aditional libraries and functions that will be needed.

In [ ]:
import cv2
import numpy
import pandas
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from tensorflow.keras import backend; backend.set_image_data_format("channels_last")
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, BatchNormalization

## 3. Introduction to pneumonia

Pneumonia is a bacterial, viral or fungal infection of the lungs that causes the alveoli to fill up with fluid or pus, which can make it hard for the oxygen to get into the bloodstream. This causes a variety of symptoms, which range from mild to severe. Pneumonia is a leading cause of hospitalization in both children and adults, affectings approximately 450 million people globally. Although antibiotics have increased survival greatly, it still remains the leading cause of death in developing countries (particularly in Southeast Asia and Africa). According to the World Health Organization (WHO), pneumonia kills 4 million people per year, half of which are children.

### Risk factors

Many factors increase the chances of getting pneumonia. Regarding the **age**, and although it can affect people of all ages, two groups have a greater risk: children under the age of 5 and adults over the age of 65. Regarding the **medical conditions**, pneumonia appears to be more common in people with chronic lung diseases (COPD, bronchiectasis, cystic fibrosis, asthma), other serious chronic illnesses (heart failure, diabetes) or a weak immune system (HIV/AIDS, chemotherapy, organ transplant, major surgery). Other risk factors are the **environment** (exposition to certain chemicals and pollutants might increase the risk) and **lifestyle habits** (smoking and drinking alcohol might increase the risk). It is also important to remark that being hospitalized, specially if a ventilator is being used for breathing, increases the risks of pneumonia. This is a quite common problem in the ICU (intensive care unit).

### Symptoms and complications

Pneumonia symptoms may include high fever, shacking chills, cough with phlegm, nausea, vomiting, chest pain or trouble breathing. If pneumonia treatment is unsuccessful, some possible complications are bacteremia (the bacterial infection spreads into the blood), lung abscesses (formation of a pus cavity in the lungs), pleural effusion (fluid accumulation around the lungs), respiratory failure, renal failure or septic shock.

### Diagnosis

Pneumonia can sometimes be difficult to diagnose because some of its symptoms are commonly seen in people with colds or flu. The diagnosis is based on medical history, physical exams and diagnostic tests. This may include a sputum test, pleural fluid culture, blood test (to see whether the immune system is fighting an infection), blood culture (to see whether the infection has spread to the bloodstream), pulse oximetry (to estimate how much oxygen is in the lungs) or bronchoscopy (to see if something is blocking the lung's airway). The best way to diagnose pneumonia, however, is through imaging. **Chest X-ray Scan** is used to look for the inflammation in the lungs while **Chest Computed Tomography Scan** is used to see how much of the lungs is being affected. Although the latter shows more detail, the former is a more widespread technique for diagnosis.

### Prevention and treatment

Although **vaccines** cannot prevent all cases of infection, they are efficient against pneumonia caused by *Streptoccocus pneumoniae* and *Influenza virus*. Other ways of preventing pneumonia are related with changing lifestyle habits. For instance, washing hands regularly, stop smoking or keeping the immune system strong through physical activity and a healthy diet can help. As for the treatment, it depends on the type of pneumonia, the germ causing the infection and how severe it is. In most cases, oral **antibiotics** are used to cure the infection and prevent complications. If this were not enough, intravenous antibiotics and oxygen therapy might be needed.

### Classification

The different types of pneumonia are named after the way in which someone gets the infection or the germ that causes such infection. The most common ones are **Community-Acquired Pneumonia** (CAP), **Hospital-Acquired Pneumonia** (HAP) and **Aspiration Pneumonia**.

## 4. Introduction to deep learning

### Deep Neural Networks (DNN)

From a very general perspective we could describe neural networks as systems composed of inputs (variables with which we will make decisions), outputs (decisions) and inner elements that we can adjust in the learning process. We feed this system with data until it is well adjusted and guesses the predictions correctly. The fundamental unit of artificial neural networks are **neurons**. These are elements that receive inputs (normalised between 0 and 1), combine them linearly (multiplying them to weights and adding them to a bias), include non-linearity and give an output. Neurons are disposed in **layers**, whose width represents the number of neurons in that layer. Several layers connected result in DNNs, whose depth is the number of layers. There are 3 types of layers:

- **Input layer**: it contains the data to analyse encoded in a certain way.
- **Hidden layer** (can be many): it processes the data through mathematical transformations.
- **Output layer**: it gives the result encoded in a certain way.

<img src="NN1.png">

An important element in the design of DNNs is the **non-linearity**. As 2 consecutive linear transformations are equivalent to a single one (see the image down below), stacking linear layers one after another would be useless. This does not happen with non-linear layers, which produce a great complexity when being stacked one after another.

<img src="NN2.png">

The non-linearity is achieved through **activation functions**, that transform complex inputs into simpler outputs. An intuitive example would be the following: the temperature can vary over a wide range of values but there is a threshold below which we put on a jacket and over which it is not necessary to do so. All activation functions that are used in deep learning are monotonic, continuous and differentiable (properties needed for optimization). Some examples are:

- **Sigmoid** (logistic function): below the threshold: ⁓ 0 ; over the threshold: ⁓ 1.
- **TanH** (hyperbolic tangent): below the threshold: ⁓ -1 ; over the threshold: ⁓1.
- **ReLu** (rectified linear unit): below the threshold: 0 ; over the threshold: x
- **Softmax**: while other activation functions get an input value and transform it, regardless of the other elements, this considers the information about the whole set of inputs we have and transforms them into a probability distribution (the outputs are in the range from 0 to 1 and their sum is exactly 1). This activation function is typically used in the output layer (it gives very intuitive results) but not in others (a lot of information about data variability is lost).

Another important element in the design of DNNs is the mechanism of propagation. **Forward propagation** is the process of pushing inputs through the network. At the end of each epoch, the obtained outputs are compared to targets to form the errors. **Backward propagation** is the process of calculating the contribution of each parameter to the errors and updating these parameters accordingly. The algorithm used for backpropagation is the **gradient descent**, which searches for an optimum in the parameters that allows reducing the error. This algorithm usually goes along with a learning rate that establishes how fast the learning process is. High learning rates can impede convergency while low learning rates can make convergency slow; due to this, learning rates are set to be dynamic and decrease as the learning process goes by (first, we want to reach the optimum region fast; then, we want to be capable of doing precise changes). This process is dependant from the initialisations that we do on parameters (weights and biases), which tends to be random. Therefore, the learning process tries with distinct initialisations (known as epochs) and gives more relevance to those working better.

<img src="NN3.png">

But how do DNNs actually learn to predict? The idea is that certain layers specialise in selecting features, others specialise in analysing them… But really, it is not that clear how the learning happens. This is a major criticism to DNNs; they are *black boxes* that give good results, but such results are difficult to interpret.

### Convolutional Neural Networks (CNN)

A CNN is a type of neural network used primarily to classify images. CNNs process images as **tensors**, which are matrices of numbers with additional dimensions (arrays nested within arrays). Specifically, they deal with 4D tensors. The dimensions of each matrix correspond to the width and height of an image. The depth represents how many colors are encoded (RGB produces an image with a depth of three layers or channels).

<img src="NN4.png">

From a mathematical point of view, a **convolution** measures how much two functions overlap as one passes over the other. With image analysis, we can think of the input image being analyzed as the static function, and the second function is what we call a filter (because it picks up a signal or feature in the image). However, not just one filter is passed over a single image, but many, and each filter picks up a different signal. CNNs take those filters and map them one by one, creating a map that indicates where each feature occurs. Let us see how the filters are applied. The CNN purpose is to find which of the initial raw numbers provided are significant signals. Rather than focusing on one pixel at a time, a CNN takes square patches of pixels and passes them through a filter, which is also a square matrix, the kernel, equal in size to the patch. The matrix that results from the dot product between the input and the kernel is known as the activation map. If the two matrices have high values in the same positions, the output will be high; if they do not, it will be low. In this way, a single value can tell us whether the pixel pattern in the input image matches the pixel pattern expressed by our filter. While doing a convolution, **zero-padding** is used to add 2 columns and 2 rows on the edges with all zeros. In this way, dimensionality is preserved.

<img src="NN5.png">

CNNs, however, perform more operations on the input images than just convolutions themselves. Some layers which are tipically required are **max pooling** (it reduces the image size by considering the larger value per patch; in this way, only those locations with stronger correlations to each feature are preserved) and **flatten** (transforming the matrix into a single columns will be necessary at some point). Additionally, **dropout** (to randomly ignore some neurons and reduce overfitting) and **batch normalization** (to stabilize the learning process and reduce the number of epochs required) could be used.

### Transfer learning

One concept that needs to consider is **transfer learning**, which consists on taking a deep neural network that is already trained in a certain problem to apply it into another problem. Although transfer learning is usually done between similar problems, it is surprisingly not always like that.

## 5. Introduction to Spark

### High-level Spark

Apache Spark is a unified analytics engine for **big data processing** that can be implemented in Python. Some interesting features about Spark are the followings: it is parallel (each executer operates on data which is local to it), lazy (execution plans are built but only when the result is needed the computation starts), chained (operations can be concatenated) and reduces the access to memory. High level Spark is based on DataFrames (two dimensional structures where each column has a specific datatype and each row contains a record) and SQL (used for querying the DataFrames).

### Machine Learning Library (MLlib)

MLlib is a Spark library that makes **machine learning** scalable and easy. In machine learning, it is common to run a sequence of algorithms to learn from data. MLlib represents such workflow as a pipeline, which consists of a sequence of stages, applied to DataFrames, to be run in a specific order. In transformer stages, the method *transform()* converts a DataFrame into another, generally by appending one or more columns. In the estimator stages, the method *fit()* takes a DataFrame and generates a model, which is itself a transformer that becomes part of the pipeline. A whole pipeline can be regarded as an estimator.

An important task in machine learning is model selection (finding the best model or the best parameters for a task). In MLlib, the model selection tools work in the following way. First, training data is split into training and validation datasets. Then, there is an iteration over the set of parameters (for each combination, they estimator is fitted with training data and the model performance is evaluated with an evaluator). Eventually, the model with the best-performing set of parameters is selected. The most efficient tool is cross-validation, where the dataset is split into folds that are used to separate training-validation pairs.

### Spark Deep Learning (SparkDL) or Deep Learning Pipelines (DLPipelines)

SparkDL or DLPipelines is an open source library created by Databricks that provides high-level APIs for scalable **deep learning**. Although it focuses on ease of use and integration, the fact that is based in TensorFlow and Keras results in no performance being sacrified. One great advantage is that DLPipelines is built by the creators of Spark, so is merged as an official API (for instance, it uses Spark Pipelines for training and Spark DataFrames for deploying models).

## 6. Dataset description

The dataset we receive is organized into 3 folders (Train, Test, Validation) and contains subfolders for each image category (**Pneumonia and Normal**). In total, there are **5,863 X-Ray images** in a JPEG format. These chest X-ray images were taken from pediatric patients of 1 to 5 years old in the Guangzhou Women and Children’s Medical Center. All chest X-ray imaging was performed as part of patients’ routine clinical care. For the analysis of chest x-ray images, they were initially screened for quality control by removing all low quality or unreadable scans. The diagnoses for the images were then graded by two expert physicians before being cleared for training the AI system. In order to account for any grading errors, the evaluation set was also checked by a third expert.

The first image correspond to a normal person. The second image correspond to a patient with pneumonia.

<img src="normal.jpeg"  width="500" height="500">
<img src="pneumonia.jpeg"  width="500" height="500">

## 7. Dataset preparation

### First approach (compatible with TensorFlow)

Although the dataset has its own separation, there is no real difference between the images in Train, Test or Validation. Therefore, we will merge all images in order to perform a segregation which we find more logical. For the moment, we will create 3 arrays. The **path_array** includes the path to all the images. The **label_array** includes the labeling for all the images (we assign 0 to "*Normal*" and 1 to "*Pneumonia*"). The **image_array** includes all the images after an standarization, which was necessary because the images had different dimensions (we set them all to 224x224 pixels) and pixel values were going from 0 to 255 (we set them all to a range between 0 and 1). These arrays are then combined in a dataframe, which will be used for defining the train and test datasets.

In [ ]:
#We define a function to create the arrays with the path and the labelling
#Images belonging to the folder "NORMAL" are given the label 0
#Images belonging to the folder "PNEUMONIA" are given the label 1

def create_arrays(folder, path_array, label_array):
    
    for image in os.listdir(folder+"/NORMAL"):
        path_array.append(folder+"/NORMAL/"+image)
        label_array.append(to_categorical(0,num_classes=2))

    for image in os.listdir(folder+"/PNEUMONIA"):
        path_array.append(folder+"/PNEUMONIA/"+image)
        label_array.append(to_categorical(1,num_classes=2))

#We define a function that standarizes all images through the following transformations:
#Resize: all the images are set to have 224x224 pixels.
#Normalize: all pixels are set to have values ranging from 0 to 1.
    
def standarize(path_array, image_array):
    
    for image in path_array:
        image = cv2.imread(str(image))
        image = cv2.resize(image, (224,224))
        image = image.astype(float)/255.
        image_array.append(image)

#We define the path to the folders

train_data="C:/Users/User/Desktop/BDE/chest_xray/train"
test_data="C:/Users/User/Desktop/BDE/chest_xray/test"
val_data="C:/Users/User/Desktop/BDE/chest_xray/val"

#We create the path_array and label_array calling the "create_array" function on the three directories

path_array = []
label_array = []
create_arrays(train_data, path_array, label_array)
create_arrays(test_data, path_array, label_array)    
create_arrays(val_data, path_array, label_array)

#We create the image_array calling the "standarize" function

image_array = []
standarize(path_array, image_array)

Now that a dataframe with all the images has been created, we have an overlook on some metric and its structure. We can see that there are 5856 images; 4273 belong to patients with pneumonia while 1583 belong to normal people. Although this ratio is not ideal, the fact that we have such a big number of samples can compensate. Down below, we show how the processed images will be seen (the image size should allow for detection of differentiating patterns).

In [ ]:
pyplot.imshow(image_array[0])

In [ ]:
pyplot.imshow(image_array[5855])

We can now create the train, validation and test datasets. The **train and validation datasets**, which will be used for training our algorithms through cross-validation, includes 90% of all the images (~ 5,300 images). From them, 90% will correspond to the **train dataset** (~ 4,800 images) and 10% to the **validation dataset** (~ 500 images). The **test dataset**, which will be used for evaluating the performance of our algorithms, includes 10% of all the images (~ 600 images). Once they are defined, we need to make sure that their ratio of "*Normal*" vs "*Pneumonia*" is similar to the one found in the original dataset.

In [ ]:
X = numpy.array(image_array)
Y = numpy.array(label_array)
temp_X, test_X, temp_Y, test_Y = train_test_split(X, Y, test_size=0.1, random_state=1)
train_X, val_X, train_Y, val_Y = train_test_split(temp_X, temp_Y, test_size=0.1, random_state=1)

In [ ]:
train_X.shape, train_Y.shape

In [ ]:
val_X.shape, val_Y.shape

In [ ]:
test_X.shape, test_Y.shape

### Second approach (compatible with SparkSQL)

The approach we had being considering turned out not to be compatible with SparkSQL. We noticed this when we tried, unsuccesfully, to convert our Pandas DataFrame into a SparkSQL DataFrame. This second approach uses a Spark built-in function to load the images in binary format. It is much simpler, but it does not allow preprocessing the images as we did before.

In [ ]:
def create_dataframe(folder):
    
    df_temp = ImageSchema.readImages(folder+"/NORMAL")
    df_temp = df_temp.withColumn("label", lit(0))
    try: df = df.union(df_temp)
    except: df = df_temp
    
    df_temp = ImageSchema.readImages(folder+"/PNEUMONIA")
    df_temp = df_temp.withColumn("label", lit(1))
    try: df = df.union(df_temp)
    except: df = df_temp
        
    return df
    
train_data="C:/Users/User/Desktop/BDE/chest_xray/train"
test_data="C:/Users/User/Desktop/BDE/chest_xray/test"
val_data="C:/Users/User/Desktop/BDE/chest_xray/val"

df_train = create_dataframe(train_data)
df_test = create_dataframe(test_data)    
df_val = create_dataframe(val_data)
df = df_train.union(df_test.union(df_val))

In [ ]:
df.count()

We can now create the train and test datasets. The **train dataset**, which will be used for training our algorithms through cross-validation, includes 90% of all the images (~ 5,300 images). The **test dataset**, which will be used for evaluating the performance of our algorithms, includes 10% of all the images (~ 600 images). Once they are defined, we need to make sure that their ratio of "*Normal*" vs "*Pneumonia*" is similar to the one found in the original dataset.

In [ ]:
train_df, test_df = df.randomSplit([0.9, 0.1])

In [ ]:
train_df.count()

In [ ]:
test_df.count()

## 8. Dataset analysis

### First approach (TensorFlow): create a CNN ad-hoc and use it as an estimator

We set the input shape. Depending on the format, the depth should be the first or the last.

In [ ]:
height=224; width=224; depth=3
if backend.image_data_format() == "channels_first":
    inputShape = (depth, height, width)
    chanDim = 1
elif backend.image_data_format() == "channels_last":
    inputShape = (height, width, depth)
    chanDim = -1

We create the network. Ours has 5 convolutional layers, mixed with activation layers (all of them using the relu function, except for the last one, which uses the softmax), batch normalization layers, max pooling layers, dropout layers and flatten and dense layers.

In [ ]:
model = Sequential()

model.add(Conv2D(32, (3, 3), padding="same", input_shape=inputShape)) 
model.add(Activation("relu")) 
model.add(BatchNormalization(axis=chanDim)) 
model.add(MaxPooling2D(pool_size=(3,3))) 
model.add(Dropout(0.25))

model.add(Conv2D(64, (3,3), padding="same")) 
model.add(Activation("relu")) 
model.add(BatchNormalization(axis=chanDim))

model.add(Conv2D(64, (3,3), padding="same")) 
model.add(Activation("relu")) 
model.add(BatchNormalization(axis=chanDim)) 
model.add(MaxPooling2D(pool_size=(2,2))) 
model.add(Dropout(0.25))

model.add(Conv2D(128, (3,3), padding = "same")) 
model.add(Activation("relu")) 
model.add(BatchNormalization(axis=chanDim))

model.add(Conv2D(128, (3,3), padding = "same")) 
model.add(Activation("relu")) 
model.add(BatchNormalization(axis=chanDim)) 
model.add(MaxPooling2D(pool_size=(2,2))) 
model.add(Dropout(0.25)) 

model.add(Flatten()) 
model.add(Dense(1024)) 
model.add(Activation("relu")) 
model.add(BatchNormalization()) 
model.add(Dropout(0.5))
model.add(Dense(2))
model.add(Activation("softmax"))

We compile the model using *categorical crossentropy* as loss function and *adam* as opitimizing function. We convert the model to JSON format and save it.

In [ ]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.fit(train_X, train_Y, batch_size=32, epochs=5, validation_data=(val_X, val_Y), verbose=1)
model_json = model.to_json()
with open("model.json", "w") as json_file: json_file.write(model_json)
model.save_weights("model.h5")

### Second approach (SparkSQL): use a pre-built and pre-trained CNN and use it as an estimator

The use of pre-built and pre-trained DNNs is a method known as "*transfer learning*". There is a SparkDL tool that takes out the last layer of pre-trained networks and uses its resulting output as input features for a new classification network. By doing so, the time needed to train the DNNs shortens and the quality of the classification improves. The workflow to implement transfer learning is the following: (I) Select a pre-trained model. (II) Truncate the last layer of the selected DNNs so that the last output produces a set of features instead of a classification. (III) Implement the truncated network in a new network for a differet dataset.  
  
Within the Keras Deep Learning dataset we find 4 networks that can be implemented to use transfer learning in CNNs:

- **VGG**: sequentian network with 3×3 convolutional layers, two dense layers (4096 nodes) and maxpooling. It is one of the first models designed for transfer learning and, due to its depth, is very slow to train, so it is often not the best option.
- **ResNet50**: network designed after VGG with some improvement in the training time. It uses a different approach: network-in-network instead of sequential architecture; which builds a very deep network from smaller modules. By doing so, this model has a great depth, as VGG, but is much smaller and therefore easier to implement.
- **Inception V3**: built from several modules that compute different features with different convolutional depths to finally converge all features into one channel to feed the next layer of the network. The V3 version was developed by Keras, and it is probably the most widely used model because it is faster than others and the accuracy of the predictions is often better.
- **Xception**: extention of the Inception model that uses multiple independent convolutions where Inception had some standard modules.  

In order to decide which model to implement we did some research on each model performance and suitability. We found that Inception V3 is the most extended model for transfer learning in image classification problems. Moreover, there is one repository in Github that, using our pneumonia dataset, had already analyzed the performance of these four models for transfer learning and compared the results. They found that using Inception V3 gives the best results and, consequently, we decided to use the Iception V3 model for our exercise.

In [ ]:
stage1 = DeepImageFeaturizer(inputCol="image", outputCol="features", modelName="InceptionV3")
stage2 = LogisticRegression(labelCol="label", featuresCol="features", predictionCol="prediction",
                            maxIter=20, regParam=0.05, elasticNetParam=0.3)
pipeline = Pipeline(stages=[stage1, stage2])
model = pipeline.fit(train_df)
test = model.transform(test_df)

In [ ]:
#Ideally, we would implement a cross validation where we could set a grid of possible parameters for the logistic regression 
# grid = ParamGridBuilder().addGrid(stage1.regParam,[0.1,0.05,0.01]).addGrid(stage1.elasticNetParam,[0.1,0.05,0.01]).build()
# evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
# validator = CrossValidator(estimator=pipeline, estimatorParamMaps=grid, evaluator=evaluator, numFolds=3)
# model = validator.fit(train_df)
# test = model.transform(test_df)

## 9. Dataset evaluation

## 10. Conclusions

## 11. Bibliography

#### Pneumonia

- https://www.nhlbi.nih.gov/health-topics/pneumonia
- https://www.mayoclinic.org/diseases-conditions/pneumonia/symptoms-causes/syc-20354204
- https://www.lung.org/lung-health-and-diseases/lung-disease-lookup/pneumonia/learn-about-pneumonia.html
- https://medlineplus.gov/pneumonia.html

#### Machine learning and Deep learning

- https://www.tensorflow.org/guide/keras/
- https://github.com/databricks/spark-deep-learning
- http://spark.apache.org/docs/latest/ml-guide.html
- https://github.com/ajaysh2193/Predicting-Pneumonia-from-X-ray-images-using-CNN-and-Keras/blob/master/README.md

#### Dataset

- https://www.kaggle.com/paultimothymooney/chest-xray-pneumonia
- https://www.cell.com/cell/fulltext/S0092-8674(18)30154-5